# Roofs
### ggpl workshop_04
![Reference](./images/ref.png)

In [1]:
from larlib import *

verts = [
    [0,0,0], #1
    [1,1,0], #2
    [2,2,0], #3
    [4,2,0], #4
    [4,1,0], #5
    [4,0,0], #6
    [1,1,3], #7
    [4,1,3]  #8
]

faces = [
    [1,7,8,6],
    [6,8,5],
    [5,8,7,2],
    [1,2,7],
    [1,6,5,2],
    
    [7,8,5,2],
    [5,8,4],
    [2,5,4,3],
    [7,2,3],
    [8,7,3,4]
]
faces = [i[::-1] for i in faces]
pol = MKPOL([verts,faces,[1]])
roof = STRUCT([
        pol,
        MAP([S2,S1,S3])(pol)
    ])
VIEW(roof)

Evaluating fenvs.py..
...fenvs.py imported in 0.01 seconds


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x6a05480> >

![First result](./images/myhpc.png)
There is a little normals discrepancy, but it is ok. 
Let's discover the functioning of the UKPOL func

In [2]:
UKPOL(roof)

[[[4.0, 0.9999998807907104, 3.000000238418579],
  [0.0, 0.0, 0.0],
  [1.0, 0.9999998807907104, 3.000000238418579],
  [4.0, 0.0, 0.0],
  [4.0, 1.0, 0.0],
  [4.0, 1.0, 3.0],
  [4.0, 0.0, -1.3866695599588098e-32],
  [4.0, 1.0, 3.0],
  [1.0, 1.0, 0.0],
  [4.0, 1.0, 0.0],
  [1.0, 1.0, 3.0],
  [1.0, 1.0, -1.268878707724281e-32],
  [1.0, 1.0, 3.0],
  [1.1920928955078125e-07, 0.0, -3.9021209105317674e-16],
  [4.0, 1.0, 0.0],
  [0.0, 0.0, 0.0],
  [1.0, 1.0, 0.0],
  [4.0, 0.0, 0.0],
  [4.0, 1.0, 3.0],
  [1.0, 1.0, 0.0],
  [1.0, 1.0, 3.0],
  [4.0, 1.0, 0.0],
  [4.0, 2.0, 0.0],
  [4.0, 1.0, 3.0],
  [4.0, 1.0, 0.0],
  [4.0, 2.0, 0.0],
  [1.0, 1.0, 0.0],
  [2.0, 2.0, 0.0],
  [4.0, 1.0, 0.0],
  [2.0, 2.0, 0.0],
  [1.0000001192092896, 1.0, 3.0],
  [1.0000001192092896, 1.0, 0.0],
  [4.0, 2.0, 0.0],
  [1.0, 1.0000001192092896, 3.000000238418579],
  [2.0, 2.0, 0.0],
  [4.0, 1.0000001192092896, 3.000000238418579],
  [0.9999998807907104, 4.0, 3.000000238418579],
  [5.960464477539063e-08, 0.0, -2.3841857910

Woah. It generates what seems to be the argument of a MKPOL function...

In [3]:
polygon_data = UKPOL(roof)
verts_data = list(set([tuple([round(x),round(y),round(z)]) for [x,y,z] in polygon_data[0]]))
verts_data

[(1.0, 1.0, 0.0),
 (4.0, 1.0, 3.0),
 (1.0, 4.0, 0.0),
 (4.0, 0.0, 0.0),
 (1.0, 1.0, 3.0),
 (2.0, 4.0, 0.0),
 (4.0, 1.0, 0.0),
 (2.0, 2.0, 0.0),
 (0.0, 0.0, 0.0),
 (4.0, 2.0, 0.0),
 (0.0, 4.0, -0.0),
 (1.0, 4.0, 3.0)]

Ok, I've got the vertices.
Now I want to get the vertices that are on the same plane.

In [68]:
def areCoplanar(v1,v2,v3,v4):
    """Returns if v1,v2,v3,v4 are coplanar."""
    v1v4 = VECTDIFF([v1,v4])
    v1v2 = VECTDIFF([v1,v2])
    v1v3 = VECTDIFF([v1,v3])

    prod = VECTPROD([v1v2,v1v3])
    return prod[0]*v1v4[0]+prod[1]*v1v4[1]+prod[2]*v1v4[2] == 0

areCoplanar(verts_data[0],verts_data[2],verts_data[3],verts_data[5])

True

I want to find the base plane.

In [8]:
def getVertsIndexesOnBasePlane(verts, basePlaneZ=0.0):
    """Returns the indexes of all the verts on the base plane"""
    ret = []
    for i in range(len(verts)):
        if round(verts[i][2]) == basePlaneZ:
            ret.append(i+1)
    return ret

print getVertsIndexesOnBasePlane(polygon_data[0])

[2, 4, 5, 7, 9, 10, 12, 14, 15, 16, 17, 18, 20, 22, 23, 25, 26, 27, 28, 29, 30, 32, 33, 35, 38, 40, 41, 43, 44, 47, 48, 50, 51, 52, 53, 54, 56, 57, 60, 61, 62, 63, 64, 65, 66, 68, 70, 71]


In [6]:
def isListInList(l1, l2):
    for i in l1:
        if not( i in l2 ):
            return False
    return True

isListInList([2,4,5],verts), isListInList([1,2,4,5],verts)

(False, False)

In [10]:
vertsIndexes = getVertsIndexesOnBasePlane(polygon_data[0])
faces = [i for i in polygon_data[1] if isListInList(i,vertsIndexes)]
print faces
VIEW(MKPOL([polygon_data[0],faces,[1]]))

[[15, 17, 16, 18], [26, 28, 27, 29], [51, 53, 52, 54], [62, 64, 63, 65]]


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x6a0fdb0> >

![Base plane](./images/base_plane.png)
OK. What I found is almost useless. Later I'll try another approach:<br>
For every face of the HPC, get all the vertices and group them in two sets:
* ***S1*** The vertices with z=0
* ***S2*** The other vertices

Given two points of S1 (if there are) called V1 and V2, consider the plane P formed by V1,V2 and a vertex with z=V1.z<br>
If even a single point in S2 is not coplanar to P, then add the face in a list.<br>
Then draw every face in the list so generated.


In [15]:
# Vertices and faces of the roof model
data = UKPOL(roof)

# Indexes of the verts with z=0.0
vertsIndexes = getVertsIndexesOnBasePlane(data[0],0.0) 

# I need to find every face with at least two but not all vertices with z=0.0
# Let's define a function that given two lists returns True only if two or more elements of the first 
#   list are in the second one:
def twoElemInList(l1, l2):
    i = 0
    for elem in l1:
        if elem in l2:
            i += 1
            if i == 2:
                return True
    return False

# Test this
twoElemInList([1,2],[1,4,5,6,2]),twoElemInList([1,2,3],[4,5,6,2])

(True, False)

In [17]:
# OK.

# Now a function to check if not every element of a list is inside another list:
def notAllInList(l1, l2):
    for elem in l1:
        if not(elem in l2):
            return True
    return False

# Test
notAllInList([1,2],[1,4,5,6,2]),notAllInList([1,2,3],[4,5,6,2])

(False, True)

In [69]:
# Seems correct.

# Let's get the list of useful faces:
faces = [i for i in data[1] if twoElemInList(i,vertsIndexes) and notAllInList(i,vertsIndexes)]
faces

[[1, 3, 2, 4],
 [6, 7, 5],
 [8, 10, 9, 11],
 [13, 14, 12],
 [19, 21, 20, 22],
 [24, 25, 23],
 [31, 32, 30],
 [33, 35, 34, 36],
 [37, 39, 38, 40],
 [42, 43, 41],
 [44, 46, 45, 47],
 [49, 50, 48],
 [55, 57, 56, 58],
 [60, 61, 59],
 [67, 68, 66],
 [69, 71, 70, 72]]

In [77]:
# Now I need to pick from this list only the faces not orthogonal to the XY plane

# To start, here a function to divide the verts of a face in two sets as described earlier:
def divideVerts(verts, basePlaneZ=0.0):
    baseVerts = []
    otherVerts = []
    for vert in verts:
        if round(vert[2]) == basePlaneZ:
            baseVerts.append(vert)
        else:
            otherVerts.append(vert)
    return baseVerts, otherVerts
    
# Let's define a function for determine if a single face is orthogonal to XY:
def isOrthoToXY(verts):
    baseVerts, otherVerts = divideVerts(verts)
    
    if len(baseVerts) < 2:
        raise ValueError
    
    # A vertex perfectly aligned with baseVerts[0] used to build an ortho plane to XY
    tmpVert = baseVerts[0][:]
    tmpVert[2] += 1
    for vert in otherVerts:
        if not areCoplanar(baseVerts[0], baseVerts[1], tmpVert, vert):
            return False
    return True
    
# Some tests:
(
    divideVerts([[1.0, 4.0, 3.0],[1.0, 4.0, 0.0],[1.0, 1.0, 0.0]]),
    isOrthoToXY([[1.0, 4.0, 3.0],[1.0, 4.0, 0.0],[1.0, 1.0, 0.0]]),
    isOrthoToXY([[0.0, 0.0, 0.0],[1.0, 0.0, 0.0],[0.0, 1.0, 3.0]])
)

(([[1.0, 4.0, 0.0], [1.0, 1.0, 0.0]], [[1.0, 4.0, 3.0]]), True, False)

In [78]:
# OK

# Here a utility func to get the vertices of a single face
def indexes2values(verts, face):
    return [verts[vert-1] for vert in face]
    
indexes2values(data[0],faces[8])

[[0.9999998807907104, 4.0, 3.000000238418579],
 [0.9999998807907104, 1.0, 3.000000238418579],
 [5.960464477539063e-08, 0.0, -2.384185791015625e-07],
 [5.960464477539063e-08, 4.0, -2.384185791015625e-07]]

In [79]:
finalFaces = [face for face in faces if not isOrthoToXY(indexes2values(data[0],face))]
finalFaces

[[1, 3, 2, 4], [33, 35, 34, 36], [37, 39, 38, 40], [69, 71, 70, 72]]

In [84]:
onlyRoof = MKPOL([data[0],finalFaces,[1]])
VIEW(onlyRoof)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x6cd52a0> >

![Only roof](./images/only_roof.png)
There is always a little normals error. Nothing important, however.<br>
Let's do some experiments with the OFFSET func:

In [87]:
VIEW(OFFSET([0.2,0.2,0.2])(SKEL_1(onlyRoof)))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x2ec9810> >

![Offset experiment](./images/offset.png)